In [5]:
import pandas as pd
import re
import glob
import os
from tqdm import tqdm
tqdm.pandas()

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import tqdm
import jpype

In [6]:
folder_path = '/Users/jaesolshin/Documents/GitHub/trendpop/'
file_path = os.path.join(folder_path, 'KPOP_comments_merged_preprocessed.csv')
comments_df = pd.read_csv(file_path)

In [7]:
# 명사추출기 객체정의
okt = Okt()

# 불용어 정의
stopwords = ['timecode', '진짜','이번','사람','정말', '뭔데', '그룹', '우리', '생각', '댓글', '느낌','계속', '지금', '최고', '영상', '처음', '축하', '대박', '이건', '당신', '제발', '항상', '아주', '다음', '정도', '모두', '보고', '그냥', '다시', '그것', '역시', '점점', '오늘', '요즘', '가장', '부분', '전부', '제일', '너머', '내용', '뭔가', '모습', '근데', '너무', '아니', '사람들', '같아요', '데리', '여러분', '세상', '자기', '다른']

# 고유명사 리스트 정의
custom_nouns = ['에스파', '조회수', '하이브', '어도어', 'BTS', '아이브', '르세라핌']

# 텍스트에서 명사만 추출하는 함수
def extract_nouns(text, stopwords=stopwords, custom_nouns=custom_nouns):
    # Okt를 통해 명사 추출
    nouns = okt.nouns(text)

    # 고유명사로 대체하는 로직
    updated_nouns = []
    for noun in nouns:
        # 고유 명사에 포함된 단어를 대체
        replaced = False
        for custom_noun in custom_nouns:
            if custom_noun in text:
                # 만약 명사가 custom_nouns에 있는 단어의 일부라면 해당 단어로 대체
                if noun in custom_noun:
                    updated_nouns.append(custom_noun)
                    replaced = True
                    break
        if not replaced:
            updated_nouns.append(noun)

    # 길이가 1 이상이면서 stopwords에 없는 명사만 남김
    final_nouns = list(set([noun for noun in updated_nouns if len(noun) > 1 and noun not in stopwords]))

    return final_nouns

# 정상작동여부 확인
print(extract_nouns("신우석 감독님 폼 미치셨네요;;ㄹㅇ 이 노래로 큐피트를 표현할 생각을 하시다니"))
print(extract_nouns("에스파 조회수 1억뷰 돌파"))

['큐피트', '신우석', '표현', '노래', '감독']
['에스파', '돌파', '조회수']


In [8]:
import os
import pandas as pd
import concurrent.futures
from tqdm import tqdm  # tqdm 함수 불러오기

def extract_nouns_batch(dataframe, folder_path, batch_size=5, initialize=True, verbose=True):
    # 디렉토리가 존재하지 않으면 생성
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 총 배치 수 계산
    batch_time = (len(dataframe) + batch_size - 1) // batch_size

    # 임시 파일 경로 설정
    temp_file_path = os.path.join(folder_path, 'processing.csv')

    # 임시 파일에 빈 데이터프레임 생성
    if not os.path.exists(temp_file_path) or initialize==True:
        temp = pd.DataFrame(columns=dataframe.columns) 
        temp.to_csv(temp_file_path, index=False)

    # 배치별 처리
    for n_try in tqdm(range(batch_time)):
        # 임시 파일에서 진행 상태 확인
        try:
            if os.path.getsize(temp_file_path) > 0:
                batch_temp = pd.read_csv(temp_file_path)
            else:
                batch_temp = pd.DataFrame(columns=dataframe.columns)  # 빈 데이터프레임 초기화
        except pd.errors.EmptyDataError:
            batch_temp = pd.DataFrame(columns=dataframe.columns)

        # 현재 배치 슬라이싱
        start_idx = n_try * batch_size
        end_idx = min((n_try + 1) * batch_size, len(dataframe))
        frac = dataframe.iloc[start_idx:end_idx]
        if verbose==True:
            print(f"Processing batch: {start_idx} to {end_idx}")

        # 명사 추출 및 처리
        frac = frac.copy()
        with concurrent.futures.ThreadPoolExecutor() as executor:
            frac['word_list'] = list(executor.map(extract_nouns, frac['comment']))

        # 배치 상태 업데이트
        if not frac.empty:
            tmp_path = os.path.join(folder_path, 'process_status.txt')
            with open(tmp_path, 'w', encoding='utf-8') as f:
                f.write(f"status : {frac.index[-1]}")  # 배치의 마지막 인덱스

        # 기존 데이터에 현재 배치 데이터 추가 (빈 행 제거 후)
        batch_temp = pd.concat([batch_temp, frac], ignore_index=True)

        # CSV 파일로 저장
        batch_temp.to_csv(temp_file_path, index=False)

In [9]:
# process_status에 저장된 마지막 인데스 읽어오기
with open('process_status.txt', 'r', encoding='utf-8') as f:
    content = f.readline()
    last_idx = int(content.split()[2])
    print(last_idx)

# last_idx가 가리키는 댓글이 proceesing.csv에 저장된 마지막 댓글과 일치하는지 확인
start_idx = last_idx +1
last_comment = comments_df.iloc[last_idx,]['comment_id_key']
if last_comment == pd.read_csv('processing2.csv').iloc[-1,]['comment_id_key']:
    print('OK - proceed to next stage')

# 배치 처리기에 넣을 데이터 프레임
comments_df2 = comments_df.iloc[start_idx:,]

636198
OK - proceed to next stage


In [10]:
# 실행 부분
import time
folder_path = '/Users/jaesolshin/Documents/GitHub/trendpop/'
extract_nouns_batch(comments_df, folder_path, batch_size=100, initialize=False, verbose=True)
print(len(pd.read_csv('processing.csv'))+len(pd.read_csv('processing2.csv')))

  0%|          | 0/119 [00:00<?, ?it/s]

Processing batch: 0 to 100


  1%|          | 1/119 [00:01<02:07,  1.08s/it]

Processing batch: 100 to 200


  2%|▏         | 2/119 [00:02<02:02,  1.05s/it]

...

Processing batch: 11700 to 11800


 99%|█████████▉| 118/119 [02:02<00:01,  1.08s/it]

Processing batch: 11800 to 11861


100%|██████████| 119/119 [02:03<00:00,  1.04s/it]


648060


In [11]:
comments_df.iloc[501999,]

Group                                                      NewJeans
Title                                                           OMG
Unnamed: 0                                                    62619
comment_type                                                  reply
comment_id_key                                                  NaN
parent_id_key                            Ugzb3AmPpDAzi_9Ced94AaABAg
comment           OMG  ETA 같이 스밍해주세요. 한 곡들으면 집계안된다함!!!! 좋아요 댓글달아...
author                                               @Bemyself_NINI
date                                           2024-04-28T16:02:13Z
likes                                                             6
Name: 501999, dtype: object